# Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Fetching Wikipedia Page using requests

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_Super_Bowl_champions'
response = requests.get(url)

# Beautiful Soup object

In [3]:
soup = BeautifulSoup(response.text, 'lxml')

# Getting Superbowl Championships table 

In [4]:
tables = soup.find_all('table')

req_table_pos = 0
for i in range(len(tables)):
    info = tables[i].contents
    for j in range(len(info)):
        if "<class 'bs4.element.Tag'>" in str(type(tables[i].contents[j])):
            try:
                if 'Super Bowl championships' in tables[i].contents[j].text:
                    req_table_pos = i
            except:
                continue
                
superbowl_table = tables[req_table_pos]

In [5]:
table_rows = superbowl_table.find('tbody').find_all('tr')

# Parsing each table row

In [6]:
file = open('rows.txt', 'w', encoding='utf-8')

for row in table_rows:
    line = ''
    
    # looping through each column in a row
    for index, item in enumerate(row.find_all('td')):
        if index in [0,1,2,3,4,5]:    # considering only required output columns
            # process each column corresponding to index nos.
            if index == 0:
                line += item.find('a').text.strip() + '|'
            elif index == 1:
                line += item.find('span').text.strip().split(', ')[1] + '|'    # split on ',' to only get year
            elif index == 2:
                line += item.find('a').text.strip() + '$'    # added '$' for future processing
                # last few rows in table are future games so 'small' tag is absent for these rows
                try:
                    line += item.find('small').text.strip() + '|'
                # break out of the loop since we don't want to consider games that are not played yet
                except:
                    break                    
            elif index == 4:
                line += item.find('a').text.strip() + '$'   
                line += item.find('small').text.strip() + '|'
            elif index == 5:
                line += item.find('span').text.strip().split('[')[0]    # split on '[' to avoid reding references part 
            else:    # handles case when index=3
                line += item.text.strip() + '|'
    
    # logic to avoid considering rows for future games that haven't been played yet
    if line.count('|') == 5:
        file.write(line+'\n')    # write to csv file
        
file.close()    

# Create DataFrame from rows.csv

In [7]:
df = pd.read_csv('rows.txt', names=['Game','Year','Winning team','Score','Losing team','Venue'],sep='|')

In [8]:
df.head()

,Game,Year,Winning team,Score,Losing team,Venue
0,I,1967,"Green Bay Packers$(1, 1–0)",35–10,"Kansas City Chiefs$(1, 0–1)",Los Angeles Memorial Coliseum
1,II,1968,"Green Bay Packers$(2, 2–0)",33–14,"Oakland Raiders$(1, 0–1)",Miami Orange Bowl
2,III,1969,"New York Jets$(1, 1–0)",16–7,"Baltimore Colts$(1, 0–1)",Miami Orange Bowl (2)
3,IV,1970,"Kansas City Chiefs$(2, 1–1)",23–7,"Minnesota Vikings$(1, 0–1)",Tulane Stadium
4,V,1971,"Baltimore Colts$(2, 1–1)",16–13,"Dallas Cowboys$(1, 0–1)",Miami Orange Bowl (3)


# Process team and venue columns to get desired output format

In [9]:
def parse_team(s):
    a = s.split('$')
    s_1 = a[0]
    s_2 = a[1]
    
    return s_1 + ' ' + s_2[1].zfill(2)

In [10]:
def parse_venue(s):
    a = s.split('(')
    s_1 = a[0]
    try:
        s_2 = a[1]
        return s_1 + s_2[0].zfill(2)
    except:
        return s_1 + ' ' + '01'

In [11]:
df['Winning team'] = df['Winning team'].apply(parse_team)
df['Losing team'] = df['Losing team'].apply(parse_team)
df['Venue'] = df['Venue'].apply(parse_venue)

In [12]:
df.head()

,Game,Year,Winning team,Score,Losing team,Venue
0,I,1967,Green Bay Packers 01,35–10,Kansas City Chiefs 01,Los Angeles Memorial Coliseum 01
1,II,1968,Green Bay Packers 02,33–14,Oakland Raiders 01,Miami Orange Bowl 01
2,III,1969,New York Jets 01,16–7,Baltimore Colts 01,Miami Orange Bowl 02
3,IV,1970,Kansas City Chiefs 02,23–7,Minnesota Vikings 01,Tulane Stadium 01
4,V,1971,Baltimore Colts 02,16–13,Dallas Cowboys 01,Miami Orange Bowl 03


# Write final results to transformed.csv

In [13]:
df.to_csv('transformed.csv', index=False)